In [1]:
import os
import numpy as np
import pandas as pd 
import glob
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, LassoCV, ElasticNetCV
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")


# start at the project dir
if 'workflow' not in os.listdir():
    os.chdir('../../../')

# setting the output dir
datadir = 'results/processed/harmonized/'
jive_dir = 'results/jive/harmonized/'
outdir = 'results/submissions/harmonized/'
os.makedirs(outdir, exist_ok=True)

In [2]:
model_dict = {'lr': LinearRegression, 
              'lasso': Lasso,
              'elastic_net': ElasticNet, 
              'lasso_cv': LassoCV,
              'elastic_net_cv': ElasticNetCV}

cmodel = 'lr'
#cmodel = 'lasso'
#cmodel = 'elastic_net'
#cmodel = 'lasso_cv'
#cmodel = 'elastic_net_cv'
cmodel_function = model_dict[cmodel]

## Loading the loading matrices

In [3]:
loadings = {} 
tpl = os.path.join(jive_dir, '*.jive-loadings.tsv')
for loading_fn in glob.glob(tpl):
    
    bn = os.path.basename(loading_fn)
    assay = bn.split('.')[0]
    loadings_matrix = pd.read_table(loading_fn)
    loadings[assay] = loadings_matrix

## Loading the input data

In [4]:
# setting up dictionaries to load data and results
train_features = {}
train_outcomes = {}

test_features = {}
test_preds = {}

#### Training Features (calculating the reduce form of each omic)

In [5]:
fn = os.path.join(jive_dir, 'common_samples.txt')
with open(fn, 'r') as f:
    common_samples = [int(x.strip()) for x in f.readlines()]

In [6]:
tpl = os.path.join(datadir, 'training_*.tsv')
for raw_fn in glob.glob(tpl):  
    
    bn = os.path.basename(raw_fn)
    assay = bn.split('.')[0].replace('training_', '')
    
    # Loading the raw matrices
    raw = pd.read_table(raw_fn, index_col=0, header=0)
    shared_columns = loadings[assay].index.tolist()
    raw = raw.loc[common_samples, shared_columns]
    raw_array = np.matrix(raw.values)
    
    ## Calculating the sample factor matrix
    loadings_array = np.matrix(loadings[assay].values)
    sample_factors = raw_array * loadings_array

    tdf = pd.DataFrame(sample_factors) 
    tdf.columns = ['{}-{}'.format(assay, i) for i in range(sample_factors.shape[1])]
    train_features[assay] = tdf
    train_features[assay].index = raw.index.tolist()

In [7]:
train_features['final'] = pd.concat([train_features['pbmc_cell_frequency'],
                                     train_features['plasma_cytokine_concentrations'],
                                     train_features['pbmc_gene_expression'],
                                     train_features['plasma_antibody_levels']], axis=1)

In [8]:
train_features['final'].head()

,pbmc_cell_frequency-0,pbmc_cell_frequency-1,pbmc_cell_frequency-2,pbmc_cell_frequency-3,pbmc_cell_frequency-4,pbmc_cell_frequency-5,pbmc_cell_frequency-6,pbmc_cell_frequency-7,pbmc_cell_frequency-8,pbmc_cell_frequency-9,...,plasma_antibody_levels-0,plasma_antibody_levels-1,plasma_antibody_levels-2,plasma_antibody_levels-3,plasma_antibody_levels-4,plasma_antibody_levels-5,plasma_antibody_levels-6,plasma_antibody_levels-7,plasma_antibody_levels-8,plasma_antibody_levels-9
31,9.083610,-0.462096,10.337036,4.260137,-32.796321,0.172084,-25.785426,4.374064,9.911269,-15.731880,...,0.036292,1.940398,-0.566566,0.246487,-0.661272,-0.321034,-1.272382,0.414521,0.255093,-0.356062
15,4.022152,4.055852,7.126219,2.782571,-21.725112,7.975466,-24.065122,6.053349,11.367983,-11.877017,...,-4.302715,-1.177101,0.286472,-2.460707,3.402857,3.517777,-3.001143,0.533420,4.192523,-1.824346
21,-1.976920,1.267354,5.679265,1.865584,-15.532700,2.905285,-16.430587,8.328064,11.241613,-1.257318,...,-4.519334,8.217305,4.983049,-0.685703,-2.091182,-1.942848,-2.940647,7.028254,4.288550,2.380629
20,-1.156982,3.179306,6.058259,3.293244,-14.595668,4.867394,-20.321299,10.406801,13.144273,-3.581408,...,4.887935,-2.813495,1.881544,-0.958646,0.116316,-0.662331,-1.721881,0.706948,0.493968,1.080263
72,2.004082,6.989833,11.409399,8.707385,-16.001405,6.525344,-24.286423,13.733439,19.011878,-10.089644,...,-4.798742,-19.567265,-2.588458,-1.951887,-6.128855,-4.283021,-4.099539,-8.174419,2.523569,-8.185369


In [9]:
fn = 'results/processed/harmonized/task_matrix.tsv'
train_outcomes = pd.read_table(fn)
train_outcomes = train_outcomes.loc[:, ['subject_id', 'igg_pt_day14', 'igg_pt_day14_fold_change',
                      'monocytes_day1', 'monocytes_day1_fold_change',
                      'ccl3_day3', 'ccl3_day3_fold_change']]

train_outcomes = train_outcomes.loc[train_outcomes.subject_id.isin(common_samples)]

fn = os.path.join(datadir, 'task_matrix.tsv')
outcomes = pd.read_table(fn)
outcomes = outcomes.loc[outcomes.subject_id.isin(common_samples)]

# remove day0
keep_cols = [x for x in outcomes.columns.tolist() if 'day0' not in x]
outcomes = outcomes[keep_cols]

# set index
outcomes.set_index('subject_id', inplace=True)

#### Testing Features

In [10]:
test_features = {} 
shared_subjects_test = set()
tpl = 'results/processed/harmonized/challenge*.tsv'

i = 0 
for raw_fn in glob.glob(tpl):  
    
    # get the assay name
    bn = os.path.basename(raw_fn)
    assay = bn.split('.')[0].replace('challenge_', '')
    
    if assay in ['abtiters']:
        continue
    
    print(raw_fn)
    print(assay)
    
    # loading the raw matrices
    raw = pd.read_table(raw_fn, index_col=0, header=0)
    raw = raw.loc[:, raw.columns.isin(loadings[assay].index)]
    
    print(raw.shape)
    
    # getting the loadings matrix 
        
    ## Calculating the sample factor matrix
    raw_array = np.matrix(raw.values)
    loadings_array = np.matrix(loadings[assay].values)
    sample_factors = raw_array * loadings_array
    
    # add to the test_features dict 
    test_features[assay] = pd.DataFrame(sample_factors)
    test_features[assay].index = raw.index
    
    
    if i == 0:
        shared_subjects_test = set(test_features[assay].index)
    else:
        shared_subjects_test = shared_subjects_test.intersection(test_features[assay].index)
    i += 1 


results/processed/harmonized\challenge_pbmc_cell_frequency.tsv
pbmc_cell_frequency
(48, 12)
results/processed/harmonized\challenge_pbmc_gene_expression.tsv
pbmc_gene_expression
(53, 58302)
results/processed/harmonized\challenge_plasma_antibody_levels.tsv
plasma_antibody_levels
(54, 31)
results/processed/harmonized\challenge_plasma_cytokine_concentrations.tsv
plasma_cytokine_concentrations
(32, 27)


### Stopped here

In [11]:
test_features.keys()

dict_keys(['pbmc_cell_frequency', 'pbmc_gene_expression', 'plasma_antibody_levels', 'plasma_cytokine_concentrations'])

In [12]:
# harmonize the samples
for assay in test_features.keys():
    
    test_features[assay] =  test_features[assay].loc[test_features[assay].index.isin(shared_subjects_test), :]

In [13]:
test_features['final'] = pd.concat([test_features['pbmc_cell_frequency'],
                                    test_features['plasma_cytokine_concentrations'],
                                    test_features['pbmc_gene_expression'], 
                                    test_features['plasma_antibody_levels']], axis=1)
test_features['final'] = test_features['final'].dropna()

In [14]:
test_features['final'].head()

,0,1,2,3,4,5,6,7,8,9,...,0,1,2,3,4,5,6,7,8,9
subject_id,,,,,,,,,,,,,,,,,,,,,
146,-0.168197,0.591133,0.080733,-0.110669,-0.316018,0.514219,-1.302908,-0.030671,0.975981,-0.156913,...,8.183297,-3.176033,0.155360,0.558878,-2.192607,1.249599,-4.345485,0.472349,1.480905,4.626724
124,0.153516,0.520815,0.138316,0.218657,-0.611031,0.171374,-0.734360,0.596164,1.085721,-0.536672,...,1.036307,-3.660369,-1.287482,0.338592,2.407526,2.267423,-3.551600,1.105633,-1.605377,-1.696955
134,0.047832,0.570368,0.086888,-0.293082,-0.346668,0.170993,-1.229557,-0.055890,1.027850,-0.273682,...,0.300551,0.499150,0.154381,-0.449342,-0.734023,-0.366694,-0.866363,-0.027360,0.595377,0.734578
132,-0.010183,0.399198,0.138870,-0.111089,-0.745567,0.372326,-0.816417,0.184654,0.959450,-0.326290,...,-2.946228,-6.788296,-0.271702,4.420225,7.838592,0.702159,-5.603234,2.623638,-3.820729,1.597873
140,0.139253,0.552296,0.340490,-0.281222,-0.289834,0.307139,-1.217472,-0.270947,1.046789,-0.356432,...,-1.927292,9.634063,-0.141657,0.725653,-4.274322,-1.329147,-1.000516,-1.211167,2.862920,3.424509


## Building Lists of Tasks Cased on Assay Type

In [15]:
tasks = pd.read_table('results/processed/harmonized/task_matrix.tsv')

tasks = tasks.loc[:, ['subject_id', 'igg_pt_day14', 'igg_pt_day14_fold_change',
                      'monocytes_day1', 'monocytes_day1_fold_change',
                      'ccl3_day3', 'ccl3_day3_fold_change']]

## Make predictions for the Ab Titers

In [16]:
ctrain_features = train_features['final']
ctest_features = test_features['final']

In [17]:
for task in train_outcomes.columns.tolist()[1:]:
    
    print(task)

    # get the outcome vector 
    ctrain_outcome = train_outcomes[['subject_id', task]]
    
    # get the shared subjects 
    shared_subjects = set(ctrain_features.index.tolist()).intersection(ctrain_outcome.subject_id)
 
    # extract the shared subjects  
    xdata = ctrain_features.loc[ctrain_features.index.isin(shared_subjects)]
    ydata = ctrain_outcome.loc[ctrain_outcome.subject_id.isin(shared_subjects)]      
    
    # impute missing values
    ydata = ydata.iloc[:, 1]
    concated_data = np.concatenate([xdata, ydata.to_frame()], axis=1)
    imputed_data = imputer.fit_transform(concated_data)
    imp_xdata = imputed_data[:, 0:-1]
    imp_ydata = imputed_data[:, -1]

    # building the model
    # use max_iter as needed
    if cmodel in ['lr']:
        lr_model = cmodel_function()
    elif cmodel in ['lasso', 'elastic_net', 'lasso_cv', 'elastic_net_cv']:
        lr_model = cmodel_function(max_iter=20000)

    # fit the model
    lr_model.fit(imp_xdata, imp_ydata)

    # make predictions for the test features
    preds = lr_model.predict(ctest_features.values)
    
    # create the ranks df
    ranks = [ctest_features.index.tolist(), np.argsort(preds)]
    ranks = list(zip(*ranks))
    ranks = pd.DataFrame(ranks, columns=['subject_id', 'rank'])
    test_preds[task] = ranks
    

igg_pt_day14
igg_pt_day14_fold_change
monocytes_day1
monocytes_day1_fold_change
ccl3_day3
ccl3_day3_fold_change


## Save predictions to the Excel File

In [18]:
form_fn = 'results/submissions/harmonized/3rdChallengeSubmissionTemplate_10032024.tsv'
form = pd.read_table(form_fn)

In [19]:
# creating a mapper between the task names for the data and the form
task_form_mapper = {'igg_pt_day14': '1.1) IgG-PT-D14-titer-Rank',
                    'igg_pt_day14_fold_change': '1.2) IgG-PT-D14-FC-Rank',
                    'monocytes_day1': '2.1) Monocytes-D1-Rank',
                    'monocytes_day1_fold_change': '2.2) Monocytes-D1-FC-Rank',
                    'ccl3_day3': '3.1) CCL3-D3-Rank',
                    'ccl3_day3_fold_change': '3.2) CCL3-D3-FC-Rank'}

In [20]:
# filling in the form
complete_form = form.copy()

In [21]:
for (task_name, form_name) in task_form_mapper.items():

    print(task_name, '-----------------', form_name)
     
    if task_name in test_preds:
    
        cranks = test_preds[task_name]

        # locate the indexes of the subjects within the form
        form_subject_indexes = form['SubjectID'].isin(cranks.subject_id.tolist())
        form_subject_indexes = form['SubjectID'][form_subject_indexes].index.tolist()

        # update the form for the current taskname
        complete_form.loc[form_subject_indexes, form_name] = cranks['rank'].astype(int)
    

igg_pt_day14 ----------------- 1.1) IgG-PT-D14-titer-Rank
igg_pt_day14_fold_change ----------------- 1.2) IgG-PT-D14-FC-Rank
monocytes_day1 ----------------- 2.1) Monocytes-D1-Rank
monocytes_day1_fold_change ----------------- 2.2) Monocytes-D1-FC-Rank
ccl3_day3 ----------------- 3.1) CCL3-D3-Rank
ccl3_day3_fold_change ----------------- 3.2) CCL3-D3-FC-Rank


In [22]:
outfn = os.path.join(outdir, 'Completed_Predictions.jive.{}.tsv'.format(cmodel))
complete_form.to_csv(outfn, sep='\t', float_format='%.0f', index=False, header=True)

In [23]:
complete_form

,SubjectID,Age,BiologicalSexAtBirth,VaccinePrimingStatus,1.1) IgG-PT-D14-titer-Rank,1.2) IgG-PT-D14-FC-Rank,2.1) Monocytes-D1-Rank,2.2) Monocytes-D1-FC-Rank,3.1) CCL3-D3-Rank,3.2) CCL3-D3-FC-Rank,4.1) IFNG/IL5-Polarization-D30-Rank
0,119,23,Female,aP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,120,27,Female,wP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,121,22,Female,aP,3.0,25.0,5.0,3.0,17.0,18.0,NaN
3,122,23,Female,aP,25.0,2.0,3.0,25.0,25.0,0.0,NaN
4,123,26,Female,wP,23.0,6.0,17.0,23.0,9.0,16.0,NaN
5,124,22,Male,aP,13.0,24.0,2.0,24.0,5.0,26.0,NaN
6,125,29,Male,wP,21.0,5.0,20.0,2.0,15.0,8.0,NaN
7,126,29,Male,wP,11.0,17.0,22.0,20.0,16.0,12.0,NaN
8,127,26,Female,aP,20.0,13.0,24.0,8.0,12.0,22.0,NaN
9,128,28,Female,wP,24.0,23.0,13.0,17.0,20.0,17.0,NaN
